# ADR1

## import modules: Run once

In [1]:
import pyvisa, serial
import sys
import time
sys.path.append('..\Instrument Driver')
import Lakeshore218, Agilent6641A, SR830, Lakeshore331, HeatSwitch

C:\ProgramData\Anaconda3\lib\site-packages\visa.py:13: FutureWarning: The visa module provided by PyVISA is being deprecated. You can replace `import visa` by `import pyvisa as visa` to achieve the same effect.

The reason for the deprecation is the possible conflict with the visa package provided by the https://github.com/visa-sdk/visa-python which can result in hard to debug situations.
  warnings.warn(


In [4]:
ser = serial.Serial('COM9')

In [5]:
ser.write(b'C')

1

In [6]:
ser.close()

## Test heat switch

In [2]:
hs = HeatSwitch.HeatSwitch('COM3')

In [8]:
hs.open()
time.sleep(5)

In [3]:
hs.close()
time.sleep(5)

In [4]:
hs.close_port()

## Test temperature reading

In [9]:
import importlib
importlib.reload(Lakeshore218)

<module 'Lakeshore218' from '..\\Instrument Driver\\Lakeshore218.py'>

In [2]:
LS1 = Lakeshore218.Lakeshore218(18)

In [3]:
LS1.IDN()

'LSCI,MODEL218S,21SA54,042109\r\n'

In [4]:
LS1.getTemperature('1')

2.778

In [5]:
LS1.getTemperature('2')

60.727

In [12]:
LS1.getInductiveVolt('4')

0.0987

In [11]:
LS2 = Lakeshore331.Lakeshore331(14)

In [7]:
LS2.IDN()

'LSCI,MODEL331S,332121,032301\r\n'

In [8]:
LS2.getTemperature('A')

18.961

## Test Lockin

In [7]:
lockin = SR830.SR830(13)

In [8]:
lockin.IDN()

'Stanford_Research_Systems,SR830,s/n55267,ver1.07 \n'

In [9]:
def configure_lockin(lockin):            
    #set parameters
    lockin.setTrig('sine')
    lockin.setSource( 'int')
    lockin.setHarm(1)
    lockin.setAmpl(0.01)  # Volts
    lockin.setFreq(13) # Hz

    lockin.setShield('ground')
    lockin.setCoupling('DC')
    lockin.setInputConfig('A-B')

    lockin.setTimeConst('1 s')
    lockin.setSlope(12)

    lockin.setSens('50 uV/pA')

    lockin.setReserve('normal')
    lockin.setFilter('both')
        
def R_to_temperature(R_volt):
    """ 
    Convert amplitude reading from lockin to temperature 
    """
    Amp = 0.01 # 0.01 V
    R_std = 10e6 # standard resistance in Ohm

    R = R_volt / (Amp / R_std)  # R: resistance of thermometer

    if R > 3590:
        Temp = 377*((1/(R-1500))+8.22e-6)**0.86296
    else:
        Temp = 4031*((1/(R-997.7))+4.54e-5)**1.15512

    return Temp

In [10]:
R_to_temperature(float(lockin.readValue('r')))

(-1.159323215111667-0.6143897360390151j)

## Test power supply

In [2]:
ps = Agilent6641A.Agilent6641A(12)

In [3]:
ps.IDN()

'HEWLETT-PACKARD,6641A,0,fA.03.00sA.01.06pA.02.01\n'

In [4]:
ps.getSetCurrent()

9.2

In [5]:
ps.getSetVoltage()

0.65

In [8]:
ps.measCurrent()

9.20164

In [7]:
ps.measVoltage()

0.626553

In [15]:
ps.getOutputStatus()

1.0

In [25]:
ps.outputOff()

In [11]:
ps.outputOn()

In [11]:
ps.setCurrent(0)

In [20]:
ps.setVoltage(0)

In [21]:
ps.is_CV_mode()

True

In [22]:
ps.is_CC_mode()

False

# ADR2

## import modules: Run once

In [1]:
import pyvisa, serial
import sys
sys.path.append('..\Instrument Driver')
import Lakeshore340, Lakeshore332, Agilent6641A, HeatSwitch

C:\ProgramData\Anaconda3\lib\site-packages\visa.py:13: FutureWarning: The visa module provided by PyVISA is being deprecated. You can replace `import visa` by `import pyvisa as visa` to achieve the same effect.

The reason for the deprecation is the possible conflict with the visa package provided by the https://github.com/visa-sdk/visa-python which can result in hard to debug situations.
  warnings.warn(


## Test heat switch

In [14]:
hs = HeatSwitch.HeatSwitch('COM3')

In [15]:
hs.open()

In [16]:
hs.close()

In [17]:
hs.close_port()

## Test temperature reading

In [2]:
LS1 = Lakeshore340.Lakeshore340(1)

In [3]:
LS1.IDN()

'LSCI,MODEL340,341911,042304\n'

In [4]:
# LS1.getTemperature('A') # 3K
import time
while True:
    print(LS1.getTemperature('A')) # 3K
    time.sleep(5)

291.678
291.678


KeyboardInterrupt: 

In [5]:
import time
while True:
    print(LS1.getTemperature('B')) # 50K
    time.sleep(5)

119.488
119.523


KeyboardInterrupt: 

In [5]:
LS2 = Lakeshore332.Lakeshore332(2)

In [6]:
LS2.IDN()

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


'LSCI,MODEL332S,332210,112002'

In [7]:
LS2.getTemperature('A') # GGG

280.34

In [8]:
LS2.getTemperature('B') # FAA

0.0

## Test power supply

In [2]:
ps = Agilent6641A.Agilent6641A(3)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3352: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  warnings.warn(


In [3]:
ps.IDN()

'HEWLETT-PACKARD,6641A,0,fA.03.00sA.01.03pA.01.06\n'

In [6]:
ps.instr.timeout

10000

In [10]:
ps.instr.write_termination

'\r\n'

In [6]:
ps.getSetCurrent()

0.0

In [11]:
ps.getSetVoltage()

0.0

In [10]:
ps.setVoltage(0)

In [5]:
ps.setCurrent(0)

In [8]:
ps.measCurrent()

-0.0319834

In [9]:
ps.measVoltage()

0.0387942

In [12]:
ps.getOutputStatus()

1.0

In [137]:
ps.outputOff()

In [136]:
ps.outputOn()

In [13]:
ps.setCurrent(9.2)

In [6]:
ps.is_CC_mode()

False

In [7]:
ps.is_CV_mode()

True

In [12]:
import time
while True:
    ps.is_CV_mode()
    time.sleep(3)

KeyboardInterrupt: 

In [2]:
import ADRMagCycle

In [4]:
from ADRMagCycle import ADR1MagCycle

In [8]:
%matplotlib qt
import h5py
import matplotlib.pyplot as plt
import numpy as np

fn= r'Z:\Logs\ADR2\ADR2 Log\2021-01\ADR2 Log-20210125.hdf5'
with h5py.File(fn,'r') as f:
    data = f['Data'][:,:]
    
#     for k,v in f.items():
#         print(k)
#         print(v)
        
#     print(f['Channels'][:])
#     print(f['Log list'][:])
    print(f['Log list'][:])
    
    
    for i,v in enumerate(f['Log list']):
        print(v)
#         if v[0]=='Pos. Induct. Volt':
        if v[0]=='Induct. Volt':
            induct_volt_index = i
            print(i)
            break
        
    print((f['Data'][-1,5]))
    

[('GGG',) ('FAA',) ('3K',) ('50K',) ('Induct. Volt',)]
('GGG',)
('FAA',)
('3K',)
('50K',)
('Induct. Volt',)
4
3e-05


In [7]:
induct_volt_index

4

In [69]:
 with h5py.File(log_filename, 'r') as f:
                
            for i,v in enumerate(f['Log list']):
                if v[0]=='Induct. Volt':
                    induct_volt_index = i
                break

In [15]:
data[3,:]

array([14.958224 ,  2.826    , 60.912    ,  0.       ,  0.       ,
        3.1695328,  2.952    ], dtype=float32)

In [71]:
import ADR1MagCycle

In [72]:
mag = ADR1MagCycle.MagCycle()

In [78]:
mag._read_latest_inductive_volt()

0.0011

In [15]:
# This is useful to check if GPIB or USB is valid.
import visa
rm = visa.ResourceManager()
rm.list_resources()
instr = rm.open_resource('USB0::0x2A8D::0x7901::SG59100109::0::INSTR')
instr.write("*IDN?")

7

In [6]:
rm.open_resource('USB0::0x2A8D::0x7901::SG59100109::0::INSTR')

<'USBInstrument'('USB0::0x2A8D::0x7901::SG59100109::0::INSTR')>

In [17]:
rm.open_resource?